In [1]:
from datetime import datetime
import os
import mysql.connector
import mysql.connector.errorcode as Error
from dotenv import load_dotenv
import requests
import streamlit as st

In [2]:
load_dotenv('.env')

True

In [3]:
def get_connection():
    conn = None
    try:
        db_config = {
        'user': os.getenv('DB_USERNAME'),
        'password': os.getenv('DB_PASSWORD'),
        'host': 'localhost',
        'database': 'birthdates'
            }
        return mysql.connector.connect(**db_config)
         
    except Error as err:
        print(f"Error: '{err}'")
    
        return conn


In [4]:
get_connection()

In [ ]:
def get_bot():
    TOKEN = os.getenv('TOKEN')
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    resp = requests.get(url).json() 
    
    if resp == {'ok': True, 'result': []}:
        return {'message': 'Nenhum bot encontrado'}
    return resp['ok']
    

In [8]:
def post_msg(name):
    TOKEN = os.getenv('TOKEN')
    CHAT_ID = os.getenv('CHAT_ID')
    url_post= f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": f"Hoje é aniversário de {name}"}

    res = requests.post(url_post, json=payload)

    print(res)


In [6]:
def get_today():
  conn = get_connection()
  today = datetime.now()
  cursor = conn.cursor()
  cursor.execute("SELECT name, date_birth FROM birth")
  resultado = cursor.fetchall()
  data = []
  for name, date_birth in resultado:
    if date_birth.month == today.month and date_birth.day == today.day:
      data.append({
        "name": name
      })
  return data


In [7]:
def birth_on(dt):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT name, date_birth FROM birth WHERE date_birth = %s", (dt,))
    resultado = cursor.fetchall()
    return resultado


In [9]:
def execute_today():
    today = get_today()
    if len(today) == 0:
        print("Estou aqui")
        return None
    print(today)
    for i in today:
        post_msg(i["name"])


In [13]:
def create_sql(name, dt):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO birth (name, date_birth) VALUES (%s, %s)", (name, dt))
    conn.commit()



create_sql('example 3', '2008-08-22')